In [1]:
# Author: Tiago Tamagusko (tamagusko@gmail.com)
# Version: 3.0 (2023-09-17)

In [2]:
import pandas as pd

In [3]:
# Data

# Structure
df = pd.read_csv('processed/ltpp_data.csv')

In [4]:
df.head()

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID
0,1989,1190.0,1985,6.1,1372.800049,12.000000,0.8984,0.8984,34_1638
1,1989,490.0,1985,3.3,260.799988,6.100000,1.2738,1.2738,16_1021
2,1989,151.0,1982,4.3,634.200012,6.000000,1.0314,1.0314,30_1001
3,1989,195.0,1984,3.2,1546.099976,16.200001,1.0448,1.0448,37_1030
4,1989,1690.0,1985,8.3,1125.199951,7.200000,0.7238,0.7238,23_1012


In [5]:
DATA = df.copy()

In [6]:
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   YEAR               395 non-null    int64  
 1   AADTT              395 non-null    float64
 2   TRAFFIC_OPEN_DATE  395 non-null    int64  
 3   SN                 395 non-null    float64
 4   PRECIPITATION      395 non-null    float64
 5   TEMPERATURE        395 non-null    float64
 6   IRI                395 non-null    float64
 7   INITIAL_IRI        395 non-null    float64
 8   STATION_ID         395 non-null    object 
dtypes: float64(6), int64(2), object(1)
memory usage: 27.9+ KB


In [7]:
unique_stations = DATA['STATION_ID'].unique()

# Creating an empty list to hold dataframes after processing each STATION_ID
dfs = []

for station in unique_stations:  # changed 'stations' to 'station' for correct iteration
    station_data = DATA[DATA['STATION_ID'] == station].sort_values('YEAR')
    min_year = station_data['YEAR'].min()
    max_year = station_data['YEAR'].max()
    
    # Creating a DataFrame with all years in the range for the specific STATION_ID
    all_years = pd.DataFrame({'YEAR': range(min_year, max_year + 1)})
    all_years = all_years.merge(station_data, on='YEAR', how='left')
    
    # Fill in the STATION_ID, TRAFFIC_OPEN_DATE, and SN values
    all_years['STATION_ID'].fillna(station, inplace=True)
    all_years['TRAFFIC_OPEN_DATE'].ffill(inplace=True)
    all_years['SN'].ffill(inplace=True)
    
    # Using interpolation for the AADTT values
    all_years['AADTT'].interpolate(inplace=True)
    
    # Compute the average values for PRECIPITATION, TEMPERATURE and THICKNESS
    all_years['PRECIPITATION'].fillna(station_data['PRECIPITATION'].mean(), inplace=True)
    all_years['TEMPERATURE'].fillna(station_data['TEMPERATURE'].mean(), inplace=True)
    
    # Using interpolation for the IRI and THICKNESS values
    all_years['IRI'] = all_years['IRI'].interpolate()
    all_years['INITIAL_IRI'] = all_years['INITIAL_IRI'].interpolate()
    # all_years['THICKNESS'] = all_years['THICKNESS'].interpolate()
    
    # Appending the processed data
    dfs.append(all_years)

# Concatenating all processed dataframes
DATA_filled = pd.concat(dfs, ignore_index=True)
DATA_filled

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID
0,1989,1190.0,1985.0,6.1,1372.800049,12.000000,0.8984,0.8984,34_1638
1,1990,1250.0,1985.0,6.1,1137.000000,13.800000,0.9420,0.8984,34_1638
2,1991,1430.0,1985.0,6.1,1059.800049,13.900000,0.9394,0.8984,34_1638
3,1992,1615.0,1985.0,6.1,921.700012,12.200000,0.8992,0.8984,34_1638
4,1993,1146.0,1985.0,6.1,1193.599976,12.900000,0.9228,0.8984,34_1638
...,...,...,...,...,...,...,...,...,...
607,2001,194.0,1984.0,5.8,517.000000,24.000000,2.5000,2.5150,15_1003
608,2002,179.5,1984.0,5.8,662.500000,24.212500,2.4820,2.5150,15_1003
609,2003,165.0,1984.0,5.8,699.700012,24.400000,2.4640,2.5150,15_1003
610,2004,280.5,1984.0,5.8,662.500000,24.212500,2.4870,2.5150,15_1003


In [8]:
# View data
def query_data(df, id):  
    return df[df['STATION_ID'] == id]

unique_station_ids = DATA['STATION_ID'].unique()
unique_station_ids

array(['34_1638', '16_1021', '30_1001', '37_1030', '23_1012', '50_1002',
       '6_1253', '18_2009', '53_1501', '56_2018', '85_1801', '85_1808',
       '26_1010', '6_2647', '26_1001', '27_1029', '28_3082', '13_4111',
       '12_9054', '12_4108', '12_4107', '12_4105', '12_4103', '12_4099',
       '12_4097', '12_3996', '17_1003', '28_3083', '1_1021', '48_3609',
       '48_3559', '48_1060', '48_1049', '47_3075', '45_1008', '40_4165',
       '40_1015', '35_2006', '35_1112', '35_1022', '12_3995', '29_1005',
       '12_1060', '28_1016', '6_8151', '1_4126', '6_2004', '6_8201',
       '5_3048', '1_4125', '15_7080', '40_1017', '15_1008', '15_1006',
       '15_1003'], dtype=object)

In [9]:
query_data(DATA_filled, '16_1021')

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID
17,1989,490.0,1985.0,3.3,260.799988,6.100000,1.2738,1.2738,16_1021
18,1990,446.0,1985.0,3.3,232.000000,7.200000,1.2409,1.2738,16_1021
19,1991,492.0,1985.0,3.3,319.700012,6.800000,1.2486,1.2738,16_1021
20,1992,402.0,1985.0,3.3,239.399994,7.600000,1.2782,1.2738,16_1021
21,1993,776.0,1985.0,3.3,430.700012,5.100000,1.2416,1.2738,16_1021
22,1994,633.0,1985.0,3.3,308.600006,7.800000,1.1898,1.2738,16_1021
23,1995,760.0,1985.0,3.3,476.600006,7.300000,1.1972,1.2738,16_1021
24,1996,605.0,1985.0,3.3,339.646156,7.023077,1.2161,1.2738,16_1021
25,1997,450.0,1985.0,3.3,429.899994,7.100000,1.2350,1.2738,16_1021
26,1998,473.0,1985.0,3.3,499.799988,7.400000,1.2292,1.2738,16_1021


In [10]:
query_data(DATA_filled, '6_8151')

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID
491,1990,2237.0,1973.0,3.5,92.199997,22.900000,0.7084,0.7084,6_8151
492,1991,2087.5,1973.0,3.5,155.699997,23.050000,1.6390,0.7084,6_8151
493,1992,1938.0,1973.0,3.5,219.199997,23.200001,2.5696,0.7084,6_8151


In [11]:
# Data engeneering
DATA_filled['AGE'] = DATA_filled['YEAR'] - DATA_filled['TRAFFIC_OPEN_DATE']

# 
# For AADTT
DATA_filled['ACCUMULATED_AADTT'] = DATA_filled.sort_values('YEAR').groupby(['STATION_ID'])['AADTT'].cumsum()

# For PRECIPITATION
# DATA_filled['MEDIAN_PRECIPITATION'] = (DATA_filled.sort_values('YEAR')
#                           .groupby(['STATION_ID'])['PRECIPITATION']
#                           .cumsum()) / (DATA_filled.sort_values('YEAR')
#                                         .groupby(['STATION_ID'])
#                                         .cumcount() + 1)

# # For TEMPERATURE
# DATA_filled['MEDIAN_TEMPERATURE'] = (DATA_filled.sort_values('YEAR')
#                         .groupby(['STATION_ID'])['TEMPERATURE']
#                         .cumsum()) / (DATA_filled.sort_values('YEAR')
#                                       .groupby(['STATION_ID'])
#                                       .cumcount() + 1)



In [12]:
DATA_filled.head()

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID,AGE,ACCUMULATED_AADTT
0,1989,1190.0,1985.0,6.1,1372.800049,12.0,0.8984,0.8984,34_1638,4.0,1190.0
1,1990,1250.0,1985.0,6.1,1137.000000,13.8,0.9420,0.8984,34_1638,5.0,2440.0
2,1991,1430.0,1985.0,6.1,1059.800049,13.9,0.9394,0.8984,34_1638,6.0,3870.0
3,1992,1615.0,1985.0,6.1,921.700012,12.2,0.8992,0.8984,34_1638,7.0,5485.0
4,1993,1146.0,1985.0,6.1,1193.599976,12.9,0.9228,0.8984,34_1638,8.0,6631.0


In [13]:
# Drop unused columns
columns_to_drop = ['TRAFFIC_OPEN_DATE', 
                   'AADTT', 
                   #'PRECIPITATION', 
                   #'TEMPERATURE'
                  ]
PROCESSED_DATA = DATA_filled.drop(columns_to_drop, axis=1)
PROCESSED_DATA.head()

,YEAR,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID,AGE,ACCUMULATED_AADTT
0,1989,6.1,1372.800049,12.0,0.8984,0.8984,34_1638,4.0,1190.0
1,1990,6.1,1137.000000,13.8,0.9420,0.8984,34_1638,5.0,2440.0
2,1991,6.1,1059.800049,13.9,0.9394,0.8984,34_1638,6.0,3870.0
3,1992,6.1,921.700012,12.2,0.8992,0.8984,34_1638,7.0,5485.0
4,1993,6.1,1193.599976,12.9,0.9228,0.8984,34_1638,8.0,6631.0


In [14]:
query_data(PROCESSED_DATA, '37_1030')

,YEAR,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID,AGE,ACCUMULATED_AADTT
45,1989,3.2,1546.099976,16.200001,1.0448,1.0448,37_1030,5.0,195.0
46,1990,3.2,1115.300049,17.400000,1.1028,1.0448,37_1030,6.0,398.0
47,1991,3.2,1397.699951,17.000000,1.1042,1.0448,37_1030,7.0,609.0
48,1992,3.2,1226.900024,16.100000,0.9996,1.0448,37_1030,8.0,818.0
49,1993,3.2,1107.900024,16.400000,1.0624,1.0448,37_1030,9.0,1037.0
50,1994,3.2,1228.900024,16.600000,1.1084,1.0448,37_1030,10.0,1235.0
51,1995,3.2,991.599976,16.400000,1.1546,1.0448,37_1030,11.0,1464.0
52,1996,3.2,1240.218178,16.445455,1.1709,1.0448,37_1030,12.0,1712.0
53,1997,3.2,990.299988,15.700000,1.1872,1.0448,37_1030,13.0,1979.0
54,1998,3.2,1296.699951,16.900000,1.1636,1.0448,37_1030,14.0,2257.0


In [15]:
# Save data
PROCESSED_DATA.to_csv('processed/data.csv', index=None, header=True)